In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np

warnings.filterwarnings("ignore")

data = pd.read_csv("M:/DataSet/usaccident/real/selectData1.csv",index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212697 entries, 0 to 212696
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   TMC                212697 non-null  int64  
 1   Severity           212697 non-null  int64  
 2   Start_Lat          212697 non-null  float64
 3   Start_Lng          212697 non-null  float64
 4   Distance(mi)       212697 non-null  float64
 5   Side               212697 non-null  int64  
 6   Temperature(F)     212697 non-null  float64
 7   Wind_Chill(F)      212697 non-null  float64
 8   Humidity(%)        212697 non-null  float64
 9   Pressure(in)       212697 non-null  float64
 10  Visibility(mi)     212697 non-null  float64
 11  Wind_Speed(mph)    212697 non-null  float64
 12  Precipitation(in)  212697 non-null  float64
 13  Amenity            212697 non-null  int64  
 14  Bump               212697 non-null  int64  
 15  Crossing           212697 non-null  int64  
 16  Gi

1.划分数据集
=

In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler

Y = data['Severity']
X = data.drop(["Severity"], axis=1)

del data

X.shape, Y.shape

((212697, 39), (212697,))

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10, stratify=Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.2,random_state=10,stratify=Y_train)

X_train.shape, X_test.shape, X_val.shape, Y_train.shape, Y_test.shape, Y_val.shape

((136125, 39), (42540, 39), (34032, 39), (136125,), (42540,), (34032,))

2.OverSampling
=

In [4]:
from collections import Counter

from sklearn.utils import resample
from sklearn.model_selection import train_test_split, StratifiedKFold,GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.tree import plot_tree
from imblearn import over_sampling

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [5]:
ros = over_sampling.RandomOverSampler(random_state=12)

X_train, Y_train = ros.fit_resample(X_train, Y_train)

print(sorted(Counter(Y_train).items()))

[(1, 111548), (2, 111548), (3, 111548), (4, 111548)]


In [6]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

3.Model
=

DTree
=

In [7]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, Y_train)

pred_dt = dtree.predict(X_test)
y_train_pred = dtree.predict(X_train[:30000])

mat_train = confusion_matrix(Y_train[:30000],y_train_pred)
mat_dt = confusion_matrix(Y_test, pred_dt)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_dt}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_dt))

del pred_dt, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [    0 24648    17    22]
 [    0     0  5064     0]
 [    0     0     0   234]]

confusion matrix :
[[    7    17     1     0]
 [   11 31721  2957   170]
 [    3  3090  4180    54]
 [    0   165    52   112]]

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00     24687
           3       1.00      1.00      1.00      5064
           4       0.91      1.00      0.96       234

    accuracy                           1.00     30000
   macro avg       0.98      1.00      0.99     30000
weighted avg       1.00      1.00      1.00     30000

              precision    recall  f1-score   support

           1       0.33      0.28      0.30        25
           2       0.91      0.91      0.91     34859
           3       0.58      0.57      0.58      7327
           4       0.33      0.34      0.34       329

    accuracy                       

In [8]:
dt_params = {
    'criterion':['gini','entropy'],
    'max_depth':[8,10,12,14,16]
}
clf = GridSearchCV(dtree, param_grid=dt_params,cv=5,scoring='recall',n_jobs=-1)
clf.fit(X_val, Y_val)


dtree = DecisionTreeClassifier(**clf.best_params_)
dtree.fit(X_train, Y_train)

pred_dt = dtree.predict(X_test)
y_train_pred = dtree.predict(X_train[:30000])

mat_train = confusion_matrix(Y_train[:30000],y_train_pred)
mat_dt = confusion_matrix(Y_test, pred_dt)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_dt}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_dt))

del pred_dt, y_train_pred

confusion matrix :
[[   13     0     2     0]
 [ 1544 11837  8693  2613]
 [  183   153  4506   222]
 [    0     0    16   218]]

confusion matrix :
[[   13     7     5     0]
 [ 2211 16487 12512  3649]
 [  326   276  6384   341]
 [    3    20    25   281]]

              precision    recall  f1-score   support

           1       0.01      0.87      0.01        15
           2       0.99      0.48      0.65     24687
           3       0.34      0.89      0.49      5064
           4       0.07      0.93      0.13       234

    accuracy                           0.55     30000
   macro avg       0.35      0.79      0.32     30000
weighted avg       0.87      0.55      0.62     30000

              precision    recall  f1-score   support

           1       0.01      0.52      0.01        25
           2       0.98      0.47      0.64     34859
           3       0.34      0.87      0.49      7327
           4       0.07      0.85      0.12       329

    accuracy                       

RFC
=

In [9]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=15)
rfc.fit(X_train,Y_train)

pred_rfc = rfc.predict(X_test)
y_train_pred = rfc.predict(X_train[:30000])

mat_rfc = confusion_matrix(Y_test, pred_rfc)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_rfc}\n")

print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_rfc))

del pred_rfc, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [   21 16941  7077   648]
 [    3   116  4856    89]
 [    0     0     1   233]]

confusion matrix :
[[    8    13     4     0]
 [   25 23182 10667   985]
 [    7   543  6572   205]
 [    0    78    25   226]]

              precision    recall  f1-score   support

           1       0.38      1.00      0.56        15
           2       0.99      0.69      0.81     24687
           3       0.41      0.96      0.57      5064
           4       0.24      1.00      0.39       234

    accuracy                           0.73     30000
   macro avg       0.51      0.91      0.58     30000
weighted avg       0.89      0.73      0.77     30000

              precision    recall  f1-score   support

           1       0.20      0.32      0.25        25
           2       0.97      0.67      0.79     34859
           3       0.38      0.90      0.53      7327
           4       0.16      0.69      0.26       329

    accuracy                       

In [10]:
rfc_params = {
    "n_estimators" : [150, 200],
    'max_depth' : [11,15,19]
}
clf = GridSearchCV(rfc, param_grid=rfc_params, scoring='recall',cv=5,n_jobs=-1)
clf.fit(X_val, Y_val)

rfc = RandomForestClassifier(**clf.best_params_)
rfc.fit(X_train,Y_train)

pred_rfc = rfc.predict(X_test)
y_train_pred = rfc.predict(X_train[:30000])

mat_rfc = confusion_matrix(Y_test, pred_rfc)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_rfc}\n")

print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_rfc))

del pred_rfc,y_train_pred

confusion matrix :
[[   15     0     0     0]
 [   59 14863  7960  1805]
 [    9   206  4655   194]
 [    0     0     7   227]]

confusion matrix :
[[   10    10     5     0]
 [   76 20707 11528  2548]
 [   19   408  6590   310]
 [    0    26    24   279]]

              precision    recall  f1-score   support

           1       0.18      1.00      0.31        15
           2       0.99      0.60      0.75     24687
           3       0.37      0.92      0.53      5064
           4       0.10      0.97      0.18       234

    accuracy                           0.66     30000
   macro avg       0.41      0.87      0.44     30000
weighted avg       0.87      0.66      0.71     30000

              precision    recall  f1-score   support

           1       0.10      0.40      0.15        25
           2       0.98      0.59      0.74     34859
           3       0.36      0.90      0.52      7327
           4       0.09      0.85      0.16       329

    accuracy                       

XGB
=

In [11]:
xgb = XGBClassifier()
xgb.fit(X_train, Y_train)
pred_xgb = xgb.predict(X_test)
y_train_pred = xgb.predict(X_train[:30000])

mat_xgb = confusion_matrix(Y_test, pred_xgb)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_xgb}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_xgb))

del pred_xgb, y_train_pred

[20:03:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
confusion matrix :
[[   15     0     0     0]
 [   18 20099  4023   547]
 [    2   173  4831    58]
 [    0     0     0   234]]

confusion matrix :
[[   11    13     1     0]
 [   26 27859  6138   836]
 [    5   431  6715   176]
 [    0    85    39   205]]

              precision    recall  f1-score   support

           1       0.43      1.00      0.60        15
           2       0.99      0.81      0.89     24687
           3       0.55      0.95      0.69      5064
           4       0.28      1.00      0.44       234

    accuracy                           0.84     30000
   macro avg       0.56      0.94      0.66     30000
weighted avg       0.91      0.84      0.86     300

In [12]:
xgb_params = {
    'max_depth' : [1,3,5],
    'n_estimators' : [150,200]
}

clf = GridSearchCV(xgb, param_grid=xgb_params,cv=5,scoring='recall')
clf.fit(X_val, Y_val)

xgb = XGBClassifier(**clf.best_params_)
xgb.fit(X_train, Y_train)
pred_xgb = xgb.predict(X_test)
y_train_pred = xgb.predict(X_train[:30000])

mat_xgb = confusion_matrix(Y_test, pred_xgb)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_xgb}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_xgb))

del pred_xgb, y_train_pred

[20:04:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:05:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:05:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:05:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[20:07:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

LGBM
=

In [13]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()
lgbm.fit(X_train, Y_train)
pred_lgbm = lgbm.predict(X_test)
y_train_pred = lgbm.predict(X_train[:30000])

mat_lgbm = confusion_matrix(Y_test, pred_lgbm)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_lgbm}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_lgbm))

del pred_lgbm, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [   34 18886  4733  1034]
 [    3   221  4700   140]
 [    0     0     0   234]]

confusion matrix :
[[   10    13     2     0]
 [   44 26378  6929  1508]
 [    8   406  6652   261]
 [    0    58    23   248]]

              precision    recall  f1-score   support

           1       0.29      1.00      0.45        15
           2       0.99      0.77      0.86     24687
           3       0.50      0.93      0.65      5064
           4       0.17      1.00      0.29       234

    accuracy                           0.79     30000
   macro avg       0.49      0.92      0.56     30000
weighted avg       0.90      0.79      0.82     30000

              precision    recall  f1-score   support

           1       0.16      0.40      0.23        25
           2       0.98      0.76      0.85     34859
           3       0.49      0.91      0.64      7327
           4       0.12      0.75      0.21       329

    accuracy                       

CAT
=

In [14]:
cat = CatBoostClassifier()
logs = cat.fit(X_train, Y_train)

pred_cat = cat.predict(X_test)
y_train_pred = cat.predict(X_train[:30000])

mat_cat = confusion_matrix(Y_test, pred_cat)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_cat}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_cat))

del pred_cat,y_train_pred

Learning rate set to 0.108006
0:	learn: 1.2549264	total: 360ms	remaining: 5m 59s
1:	learn: 1.1592293	total: 560ms	remaining: 4m 39s
2:	learn: 1.0855182	total: 761ms	remaining: 4m 12s
3:	learn: 1.0265255	total: 963ms	remaining: 3m 59s
4:	learn: 0.9772392	total: 1.15s	remaining: 3m 49s
5:	learn: 0.9365326	total: 1.35s	remaining: 3m 44s
6:	learn: 0.9029306	total: 1.55s	remaining: 3m 40s
7:	learn: 0.8703710	total: 1.78s	remaining: 3m 40s
8:	learn: 0.8414228	total: 1.99s	remaining: 3m 39s
9:	learn: 0.8188985	total: 2.17s	remaining: 3m 35s
10:	learn: 0.7966159	total: 2.39s	remaining: 3m 35s
11:	learn: 0.7768995	total: 2.6s	remaining: 3m 33s
12:	learn: 0.7610146	total: 2.79s	remaining: 3m 32s
13:	learn: 0.7426293	total: 3.02s	remaining: 3m 32s
14:	learn: 0.7287418	total: 3.19s	remaining: 3m 29s
15:	learn: 0.7179950	total: 3.39s	remaining: 3m 28s
16:	learn: 0.7042700	total: 3.59s	remaining: 3m 27s
17:	learn: 0.6943352	total: 3.77s	remaining: 3m 25s
18:	learn: 0.6870941	total: 3.95s	remaining: 

158:	learn: 0.3371889	total: 30.4s	remaining: 2m 40s
159:	learn: 0.3366793	total: 30.6s	remaining: 2m 40s
160:	learn: 0.3358498	total: 30.7s	remaining: 2m 40s
161:	learn: 0.3351755	total: 30.9s	remaining: 2m 40s
162:	learn: 0.3343550	total: 31.1s	remaining: 2m 39s
163:	learn: 0.3338610	total: 31.3s	remaining: 2m 39s
164:	learn: 0.3329108	total: 31.5s	remaining: 2m 39s
165:	learn: 0.3323675	total: 31.7s	remaining: 2m 39s
166:	learn: 0.3314463	total: 31.9s	remaining: 2m 38s
167:	learn: 0.3305038	total: 32.1s	remaining: 2m 38s
168:	learn: 0.3301856	total: 32.2s	remaining: 2m 38s
169:	learn: 0.3297206	total: 32.4s	remaining: 2m 38s
170:	learn: 0.3292471	total: 32.6s	remaining: 2m 38s
171:	learn: 0.3288550	total: 32.8s	remaining: 2m 37s
172:	learn: 0.3283450	total: 33s	remaining: 2m 37s
173:	learn: 0.3274356	total: 33.2s	remaining: 2m 37s
174:	learn: 0.3266799	total: 33.4s	remaining: 2m 37s
175:	learn: 0.3261949	total: 33.5s	remaining: 2m 37s
176:	learn: 0.3253977	total: 33.7s	remaining: 2m

314:	learn: 0.2655314	total: 59.5s	remaining: 2m 9s
315:	learn: 0.2652387	total: 59.7s	remaining: 2m 9s
316:	learn: 0.2649426	total: 59.8s	remaining: 2m 8s
317:	learn: 0.2646675	total: 1m	remaining: 2m 8s
318:	learn: 0.2644566	total: 1m	remaining: 2m 8s
319:	learn: 0.2640262	total: 1m	remaining: 2m 8s
320:	learn: 0.2636075	total: 1m	remaining: 2m 8s
321:	learn: 0.2633457	total: 1m	remaining: 2m 8s
322:	learn: 0.2626963	total: 1m 1s	remaining: 2m 7s
323:	learn: 0.2622604	total: 1m 1s	remaining: 2m 7s
324:	learn: 0.2619528	total: 1m 1s	remaining: 2m 7s
325:	learn: 0.2617820	total: 1m 1s	remaining: 2m 7s
326:	learn: 0.2614922	total: 1m 1s	remaining: 2m 7s
327:	learn: 0.2612499	total: 1m 1s	remaining: 2m 6s
328:	learn: 0.2610521	total: 1m 2s	remaining: 2m 6s
329:	learn: 0.2608663	total: 1m 2s	remaining: 2m 6s
330:	learn: 0.2607394	total: 1m 2s	remaining: 2m 6s
331:	learn: 0.2605505	total: 1m 2s	remaining: 2m 6s
332:	learn: 0.2602470	total: 1m 2s	remaining: 2m 5s
333:	learn: 0.2601933	total

469:	learn: 0.2284719	total: 1m 28s	remaining: 1m 39s
470:	learn: 0.2282354	total: 1m 28s	remaining: 1m 39s
471:	learn: 0.2278950	total: 1m 28s	remaining: 1m 39s
472:	learn: 0.2278357	total: 1m 29s	remaining: 1m 39s
473:	learn: 0.2275472	total: 1m 29s	remaining: 1m 39s
474:	learn: 0.2273864	total: 1m 29s	remaining: 1m 38s
475:	learn: 0.2270179	total: 1m 29s	remaining: 1m 38s
476:	learn: 0.2268442	total: 1m 29s	remaining: 1m 38s
477:	learn: 0.2267553	total: 1m 30s	remaining: 1m 38s
478:	learn: 0.2266543	total: 1m 30s	remaining: 1m 38s
479:	learn: 0.2265145	total: 1m 30s	remaining: 1m 37s
480:	learn: 0.2263966	total: 1m 30s	remaining: 1m 37s
481:	learn: 0.2262887	total: 1m 30s	remaining: 1m 37s
482:	learn: 0.2262254	total: 1m 30s	remaining: 1m 37s
483:	learn: 0.2260033	total: 1m 31s	remaining: 1m 37s
484:	learn: 0.2259439	total: 1m 31s	remaining: 1m 36s
485:	learn: 0.2257505	total: 1m 31s	remaining: 1m 36s
486:	learn: 0.2256640	total: 1m 31s	remaining: 1m 36s
487:	learn: 0.2254815	total:

622:	learn: 0.2048976	total: 1m 57s	remaining: 1m 10s
623:	learn: 0.2045508	total: 1m 57s	remaining: 1m 10s
624:	learn: 0.2044068	total: 1m 57s	remaining: 1m 10s
625:	learn: 0.2042940	total: 1m 57s	remaining: 1m 10s
626:	learn: 0.2041283	total: 1m 57s	remaining: 1m 10s
627:	learn: 0.2040098	total: 1m 58s	remaining: 1m 10s
628:	learn: 0.2038633	total: 1m 58s	remaining: 1m 9s
629:	learn: 0.2037337	total: 1m 58s	remaining: 1m 9s
630:	learn: 0.2036097	total: 1m 58s	remaining: 1m 9s
631:	learn: 0.2035406	total: 1m 58s	remaining: 1m 9s
632:	learn: 0.2034606	total: 1m 59s	remaining: 1m 9s
633:	learn: 0.2033977	total: 1m 59s	remaining: 1m 8s
634:	learn: 0.2031462	total: 1m 59s	remaining: 1m 8s
635:	learn: 0.2029321	total: 1m 59s	remaining: 1m 8s
636:	learn: 0.2028300	total: 1m 59s	remaining: 1m 8s
637:	learn: 0.2027713	total: 2m	remaining: 1m 8s
638:	learn: 0.2026873	total: 2m	remaining: 1m 7s
639:	learn: 0.2026241	total: 2m	remaining: 1m 7s
640:	learn: 0.2024370	total: 2m	remaining: 1m 7s
641

779:	learn: 0.1880467	total: 2m 26s	remaining: 41.3s
780:	learn: 0.1880051	total: 2m 26s	remaining: 41.1s
781:	learn: 0.1879714	total: 2m 26s	remaining: 40.9s
782:	learn: 0.1879453	total: 2m 27s	remaining: 40.7s
783:	learn: 0.1878368	total: 2m 27s	remaining: 40.6s
784:	learn: 0.1877993	total: 2m 27s	remaining: 40.4s
785:	learn: 0.1877200	total: 2m 27s	remaining: 40.2s
786:	learn: 0.1876000	total: 2m 27s	remaining: 40s
787:	learn: 0.1875384	total: 2m 27s	remaining: 39.8s
788:	learn: 0.1874744	total: 2m 28s	remaining: 39.6s
789:	learn: 0.1873690	total: 2m 28s	remaining: 39.4s
790:	learn: 0.1873264	total: 2m 28s	remaining: 39.2s
791:	learn: 0.1872219	total: 2m 28s	remaining: 39.1s
792:	learn: 0.1870682	total: 2m 28s	remaining: 38.9s
793:	learn: 0.1870278	total: 2m 29s	remaining: 38.7s
794:	learn: 0.1869187	total: 2m 29s	remaining: 38.5s
795:	learn: 0.1868272	total: 2m 29s	remaining: 38.3s
796:	learn: 0.1867842	total: 2m 29s	remaining: 38.1s
797:	learn: 0.1867162	total: 2m 29s	remaining: 3

936:	learn: 0.1755972	total: 2m 56s	remaining: 11.9s
937:	learn: 0.1755343	total: 2m 57s	remaining: 11.7s
938:	learn: 0.1754256	total: 2m 57s	remaining: 11.5s
939:	learn: 0.1753920	total: 2m 57s	remaining: 11.3s
940:	learn: 0.1752909	total: 2m 57s	remaining: 11.1s
941:	learn: 0.1751290	total: 2m 57s	remaining: 10.9s
942:	learn: 0.1750164	total: 2m 58s	remaining: 10.8s
943:	learn: 0.1749837	total: 2m 58s	remaining: 10.6s
944:	learn: 0.1749046	total: 2m 58s	remaining: 10.4s
945:	learn: 0.1748160	total: 2m 58s	remaining: 10.2s
946:	learn: 0.1747298	total: 2m 58s	remaining: 10s
947:	learn: 0.1746782	total: 2m 59s	remaining: 9.82s
948:	learn: 0.1745563	total: 2m 59s	remaining: 9.63s
949:	learn: 0.1745046	total: 2m 59s	remaining: 9.45s
950:	learn: 0.1744149	total: 2m 59s	remaining: 9.26s
951:	learn: 0.1743608	total: 2m 59s	remaining: 9.07s
952:	learn: 0.1742677	total: 3m	remaining: 8.88s
953:	learn: 0.1741181	total: 3m	remaining: 8.69s
954:	learn: 0.1740389	total: 3m	remaining: 8.5s
955:	lea